We set some variables within our environment for easy access to settings.

In [ ]:
%env THREADS 12

We assume conda is installed properly and the user has write access to create and run an environment.

We also assume enough available resources for the run. The major one is 64gb or memory. 

The rest of the parameter files assume a 12 thread CPU (the one we had for the project).

We setup an environment and apply some must needed fixes to all the applicable software:

In [ ]:
# !conda create -n l1em python=3.11 openjdk=21 -y
# !conda activate l1em
%conda config --add channels defaults
%conda config --prepend channels bioconda
%conda config --prepend channels conda-forge
%conda install -y python=3.11 openjdk=21 bwa samtools flux-simulator numpy scipy sra-tools trim-galore pysam bedtools pigz wget ipython=8.14 scikit-learn

# Edit the flux-simulator caller script to overcome safety limitation due to old age coding standards (java code reflection)
!if ! grep -q -- '--add-opens java.base/java.util=ALL-UNNAMED' $CONDA_PREFIX/share/flux-simulator-1.2.1-3/bin/flux-simulator; then sed -i '/^java -Xmx\$FLUX_MEM.*/a --add-opens java.base/java.util=ALL-UNNAMED \\\n--add-opens java.desktop/java.awt.font=ALL-UNNAMED \\\n--add-opens java.base/java.text=ALL-UNNAMED' $CONDA_PREFIX/share/flux-simulator-1.2.1-3/bin/flux-simulator; fi

# Install beers2 and camparee (which is automaticall installed with beers2 and must not be installed separetely)
# %pip install git+https://github.com/itmat/BEERS2

# Download the L1EM project from git
!git clone https://github.com/geoleven/L1EM.git


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package pygtftk-1.0.1-py36hebb334c_0 requires python >=3.6,<3.7.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pygtftk is installable with the potential options
│  ├─ pygtftk [1.0.1|1.0.2|...|1.1.0] would require
│  │  └─ python >=3.6,<3.7.0a0 , which can be installed;
│  ├─ pygtftk 1.1.0 would require
│  │  └─ python >=3.7,<3.8.0a0 , which can be installed;
│  ├─ pygtftk [1.1.1|1.1.2|...|1.2.7] would require
│  │  ├─ python >=3.6,<3.7.0a0 , which can be installed;
│  │  └─ python_abi 3.6.* *_cp36m, which can be installed;
│  ├─ pygtftk [1.1.1|1

We go ahead and download the proper experiment data and the reference genomes, while also producing the relevant files from each download:

In [95]:
%%bash
mkdir -p ./data/experiment
if [[ ! -f ./data/experiment/SRR3997504_1.fastq || ! -f ./data/experiment/SRR3997504_2.fastq ]]; then
    echo "One or both FASTQ files do not exist in the data folder. Running fasterq-dump..."
    fasterq-dump -O ./data/experiment SRR3997504
else
    echo "Both FASTQ files already exist in the data folder. Skipping fasterq-dump."
fi
if [[ ! -f ./data/experiment/SRR3997504_2_val_2_fastqc.html ]]; then
    "No trimmed files by trim_galore found. Running trim_galore..."
    trim_galore -j $((THREADS/4)) --paired --fastqc -o ./data/experiment/ data/experiment/SRR3997504_1.fastq ./data/experiment/SRR3997504_2.fastq
else
    echo "Trimmed files by trim_galore already exist. Skipping trim_galore."
fi

#https://zenodo.org/records/5146236/files/star.tar.gz?download=1


mkdir -p ./data/genome
if [[ ! -f ./data/genome/hg38.fa ]]; then
    echo "USCS genome 38 does not exist in the data folder. Downloading, unziping and indexing it..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/genome/ http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
    pigz -cd -p $THREADS ./data/genome/hg38.fa.gz > ./data/genome/hg38.fa
    bwa index ./data/genome/hg38.fa
    rm -rf ./data/genome/hg38.fa.gz
else
    echo "USCS genome 38 found. Assuming it is bwa indexed. Skipping downlaoding, unzipping and indexing it."
fi


if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ]]; then 
    echo "Downloading and unzipping Ensembl genome..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/ http://itmat.data-simulators.s3.amazonaws.com/BEERS2/CAMPAREE_RESOURCE_FILES/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz
    pigz -cd -p $THREADS ./data/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz | tar xf - -C ./data/
    rm -rf ./data/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz
else
    echo "Ensembl genome exists. Skipping downloading it."
fi
if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa.fai ]]; then 
    echo "Indexing the Ensembl genome file..."
    bwa index ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa
else
    echo "Ensebml genome file was indexed. Skipping indexing it."
fi
if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/star_index.genome/Genome ]]; then 
    echo "Downloading and unzipping star index genome..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/ http://itmat.data-simulators.s3.amazonaws.com/BEERS2/CAMPAREE_RESOURCE_FILES/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz
    pigz -cd -p $THREADS ./data/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz | tar xf - -C ./data/
    rm -rf ./data/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz
else
    echo "Start index genome already exists. Skipping downloading it."
fi 


if [[ ! -f ./data/experiment/SRR3997504.sam ]]; then
    hisat2-build --quiet -p 6 ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs
    hisat2 --quiet -x ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs -1 ./data/experiment/SRR3997504_1.fastq -2 ./data/experiment/SRR3997504_2.fastq -S ./data/experiment/SRR3997504.sam -p 6
fi
if [[ ! -f ./data/experiment/SRR3997504.bam ]]; then
    samtools view -bS -o ./data/experiment/SRR3997504.bam ./data/experiment/SRR3997504.sam
fi
if [[ ! -f ./data/experiment/SRR3997504_sorted.bam ]]; then
    samtools sort ./data/experiment/SRR3997504.bam > ./data/experiment/SRR3997504_sorted.bam
fi
if [[ ! -f ./data/experiment/SRR3997504_sorted.bam.bai ]]; then
    samtools index ./data/experiment/SRR3997504_sorted.bam
fi

Both FASTQ files already exist in the data folder. Skipping fasterq-dump.
Trimmed files by trim_galore already exist. Skipping trim_galore.
Humane genome 38 found. Assuming it is bwa indexed. Skipping downlaoding, unzipping and indexing it.
Homo sapiens genome exists. Skipping downloading it.
Indexing the Ensembl genome file...


[bwa_index] Pack FASTA... 16.74 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6199501436, availableWord=448219384
[BWTIncConstructFromPacked] 10 iterations done. 99999996 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999996 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999996 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999996 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999996 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999996 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999996 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999996 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999996 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999996 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

Start index genome already exists. Skipping downloading it.


We then skip the step of running the generate_L1EM_fasta_and_index.sh by running the commands to adjust for proper paths and skip it if it is already done.

In [99]:
%%bash
if [[ ! -f ./L1EM/annotation/L1EM.400.fa ]]; then 
    echo "Creating L1EM.400.fa from L1EM.400.bed..."
    bedtools getfasta -s -name -fi ./data/genome/hg38.fa -bed ./L1EM/annotation/L1EM.400.bed > ./L1EM/annotation/L1EM.400.fa
else
    echo "L1EM.400.fa already exists. Skipping its creation."
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.fa.sa ]]; then 
    echo "Indexing the L1EM.400.fa..."
    bwa index ./L1EM/annotation/L1EM.400.fa
else
    echo "L1EM.400.fa was already indexed. Skipping its indexing."
fi


if [[ ! -f ./L1EM/annotation/L1EM.400.ens.bed ]]; then 
    sed -E 's/^(chr)([^[:space:]]+)/\2/' ./L1EM/annotation/L1EM.400.bed > ./L1EM/annotation/L1EM.400.ens.bed
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.ens.fa ]]; then 
    echo "Creating L1EM.400.ens.fa from L1EM.400.ens.bed..."
    bedtools getfasta -s -name -fi ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa -bed ./L1EM/annotation/L1EM.400.ens.bed > ./L1EM/annotation/L1EM.400.ens.fa
else
    echo "L1EM.400.ens.fa already exists. Skipping its creation."
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.ens.fa.sa ]]; then 
    echo "Indexing the L1EM.400.ens.fa..."
    bwa index ./L1EM/annotation/L1EM.400.ens.fa
else
    echo "L1EM.400.ens.fa was already indexed. Skipping its indexing."
fi


if [[ ! -f ./L1EM/annotation/L1EM.400.ens.noY.bed ]]; then 
    awk '$1 != "Y" && $1 != "chrY"' ./L1EM/annotation/L1EM.400.ens.bed > ./L1EM/annotation/L1EM.400.ens.noY.bed
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.ens.noY.fa ]]; then 
    echo "Creating L1EM.400.ens.noY.fa from L1EM.400.ens.noY.bed..."
    bedtools getfasta -s -name -fi ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa -bed ./L1EM/annotation/L1EM.400.ens.noY.bed > ./L1EM/annotation/L1EM.400.ens.noY.fa
else
    echo "L1EM.400.ens.noY.fa already exists. Skipping its creation."
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.ens.noY.fa.sa ]]; then 
    echo "Indexing the L1EM.400.ens.noY.fa..."
    bwa index ./L1EM/annotation/L1EM.400.ens.noY.fa
else
    echo "L1EM.400.ens.noY.fa was already indexed. Skipping its indexing."
fi

L1EM.400.fa already exists. Skipping its creation.
L1EM.400.fa was already indexed. Skipping its indexing.
L1EM.400.ens.fa already exists. Skipping its creation.
L1EM.400.ens.fa was already indexed. Skipping its indexing.
Creating L1EM.400.ens.noY.fa from L1EM.400.ens.noY.bed...
Indexing the L1EM.400.ens.noY.fa...


[bwa_index] Pack FASTA... 1.04 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=494063968, availableWord=46763976
[BWTIncConstructFromPacked] 10 iterations done. 75938048 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 141444032 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 199661936 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 251402128 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 297384928 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 338250528 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 374567952 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 406842944 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 435525024 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 461013680 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 483

We then have to generate the appropriate annotation files. However, as pygtftk is not updated and has conflicting requirements with the rest of the project we are going to be providing the resulting file from runnning the command:

In [ ]:
#!gtftk bed_to_gtf -i ./L1EM/annotation/L1EM.400.bed -o ./annotations/original.gtf

We then continue to do various edits that are needed for the various runs that we are going to do. We also have to remove the colons (:) as Camparee is not compatible with them althouth they are not against the gtf standard.

In [140]:
%%bash
# Make the colons into underscores
if [[ ! -f ./annotations/nocolon.gtf ]]; then 
    sed -E 's/(\.chr([1-9]|1[0-9]|2[0-2]|x|y|mt|un)):/\1_/gI' ./annotations/original.gtf > ./annotations/nocolon.gtf
fi
# Prepare for the camparee format
if [[ ! -f ./annotations/exons.gtf ]]; then 
    sed 's/\ttranscript\t/\texon\t/g' ./annotations/nocolon.gtf > ./annotations/exons.gtf
fi
mkdir -p ./annotations/camparee
# if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz ]]; then 
#     wget -q --show-progress --progress=bar:force:noscroll -P ./data/HomoSapiens_GRCh38_Ensemblv99/ ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
# fi
# if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/Homo_sapiens.GRCh38.99.gtf.gz ]]; then 
#     wget -q --show-progress --progress=bar:force:noscroll -P ./data/HomoSapiens_GRCh38_Ensemblv99/ ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz
# fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt ]]; then 
    $CONDA_PREFIX/lib/python3.11/site-packages/camparee/bin/format_reference_files_for_camparee.py -n HomoSapiens_hg38 -o ./annotations/camparee/ -g ./data/genome/hg38.fa -a ./annotations/exons.gtf -p
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt ]]; then 
    # sed -E 's/^(chr)([1-9]|1[0-9]|2[0-2]|x|y|mt|un)(\t)/\3_/' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt
    sed -E 's/^(chr)([^[:space:]]+)/\2/' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt ]]; then 
    sort -k1,1 -k4,4n -k5,5n ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt ]]; then 
    grep -E '^#|L[^ \t]*\.1\.' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt
    python ./src/annotation400.py ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.400.txt
fi

cp ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.400.txt ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_hg38.annotation.ashes.txt
cp ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_hg38.annotation.mixed.txt

We will try to run the flux-simulator now.

To do so we firstly need to create a temporary file for it, split our fasta file into different transcript files and run it.

In [ ]:
%%bash
mkdir -p /tmp/fluxsim
flux-simulator -p ./data/flux/flux.par -x -l -s


Then we want to do 2 different runs of camparee (following the paper) to simualte the runs.

In [142]:
%%bash
if [[ ! -f ./output/run_1/CAMPAREE/data/sample1/molecule_file.txt ]]; then
    rm -rf ./output/run_1
    camparee -c ./configs/camparee.ashes.yaml -r1
fi
# if [[ ! -f ./output/run_2/CAMPAREE/data/sample1/molecule_file.txt ]]; then 
#     rm -rf ./output/run_2
#     camparee -c ./configs/camparee.mixed.yaml -r2
# fi

Running CAMPAREE using the serial job scheduler.
With the following default scheduler parameters:
 	-default_memory_in_mb : 26000
	-default_submission_args : None
	-default_num_processors : 10


And a maximum job resubmission limit of 3.
Execution of the Expression Pipeline Started...
Running jobs:0 | Pending jobs:5 | Resub jobs:0 | Completed jobs:0
--Check 5 pending jobs for satisfied dependencies:
	Submitting GenomeAlignmentStep command to serial for sample SRR3997504.
	Finished submitting GenomeAlignmentStep command to serial for sample SRR3997504.
Running jobs:0 | Pending jobs:4 | Resub jobs:0 | Completed jobs:1
--Check 4 pending jobs for satisfied dependencies:
	Submitting GenomeBamIndexStep command to serial for sample SRR3997504.
	Finished submitting GenomeBamIndexStep command to serial for sample SRR3997504.
Running jobs:0 | Pending jobs:3 | Resub jobs:0 | Completed jobs:2
--Check 3 pending jobs for satisfied dependencies:
	Submitting VariantsFinderStep command to serial for sample SRR3997504.
	Finished submitting VariantsFinderStep command to serial for sample SRR3997504.
	Submitting IntronQuantificationStep command to serial for sample SRR3997504.
	Finished submittin

We then use beer2 to account for the errors

In [143]:
# %%bash
# %conda install -y numpy=1.25.2
# %pip install git+https://github.com/itmat/BEERS2
# if [[ ! -f ./output/beers2ashes/results/S1_L2.bam ]]; then 
!beers --configfile ./configs/beers2.ashes.yaml --jobs 4 --directory ./output/beers2ashes
# fi
# if [[ ! -f ./output/beers2mixed/results/S1_L2.bam ]]; then 
# !beers --configfile ./configs/beers2.mixed.yaml --jobs 4 --directory ./output/beers2mixed
# fi

Creating specified working directory ./output/beers2ashes.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                                          count    min threads    max threads
-----------------------------------------  -------  -------------  -------------
all                                              1              1              1
create_cluster_packet                            1              1              1
create_sequencer_outputs_sam_or_bam              1              1              1
run_library_prep_packet_from_distribution        1              1              1
sequence_cluster_packet                          1              1              1
total                                            5              1              1

Select jobs to execute...

[Thu Jul 10 14:02:45 2025]
rule run_library_prep_packet_from_distribution:
    input: ../../output/run_1/CAMPAREE/data/sample1/custom_ge

We then take the output of the beers and we merge the two output files.

In [ ]:
%%bash

if [[ ! -f ./output/beers2ashes/results/S1_L1.sorted.bam ]]; then 
    samtools sort ./output/beers2ashes/results/S1_L1.bam > ./output/beers2ashes/results/S1_L1.sorted.bam
fi
if [[ ! -f ./output/beers2ashes/results/S1_L2.sorted.bam ]]; then 
    samtools sort ./output/beers2ashes/results/S1_L2.bam > ./output/beers2ashes/results/S1_L2.sorted.bam
fi
if [[ ! -f ./output/beers2ashes/results/merged.bam ]]; then 
    samtools merge -o ./output/beers2ashes/results/merged.bam ./output/beers2ashes/results/S1_L1.sorted.bam ./output/beers2ashes/results/S1_L2.sorted.bam
fi
if [[ ! -f ./output/beers2ashes/results/merged.sorted.bam ]]; then 
    samtools sort ./output/beers2ashes/results/merged.bam > ./output/beers2ashes/results/merged.sorted.bam
fi
if [[ ! -f ./output/beers2ashes/results/merged.sorted.bam.bai ]]; then 
    echo "Indexing merged results from beers (ashes)..."
    samtools index ./output/beers2ashes/results/merged.sorted.bam
fi

# if [[ ! -f ./output/beers2mixed/results/S1_L1.sorted.bam ]]; then 
#     samtools sort ./output/beers2mixed/results/S1_L1.bam > ./output/beers2mixed/results/S1_L1.sorted.bam
# fi
# if [[ ! -f ./output/beers2mixed/results/S1_L2.sorted.bam ]]; then 
#     samtools sort ./output/beers2mixed/results/S1_L2.bam > ./output/beers2mixed/results/S1_L2.sorted.bam
# fi
# if [[ ! -f ./output/beers2mixed/results/merged.bam ]]; then 
#     samtools merge -o ./output/beers2mixed/results/merged.bam ./output/beers2mixed/results/S1_L1.sorted.bam ./output/beers2mixed/results/S1_L2.sorted.bam
# fi
# if [[ ! -f ./output/beers2mixed/results/merged.sorted.bam ]]; then 
#     samtools sort ./output/beers2mixed/results/merged.bam > ./output/beers2mixed/results/merged.sorted.bam
# fi
# if [[ ! -f ./output/beers2mixed/results/merged.sorted.bam.bai ]]; then 
#     echo "Indexing merged results from beers (mixed)..."
#     samtools index ./output/beers2mixed/results/merged.sorted.bam
# fi

Indexing merged results from beers (ashes)...


In [ ]:
%pip install --upgrade torch git+https://github.com/itmat/BEERS2 jax numpyro
%conda install pysam=0.23.3

  Cloning https://github.com/itmat/BEERS2 to /tmp/pip-req-build-mk7vsj4b
  Running command git clone --filter=blob:none --quiet https://github.com/itmat/BEERS2 /tmp/pip-req-build-mk7vsj4b
  Resolved https://github.com/itmat/BEERS2 to commit 0fdd2f7db38c1c6858aab415dc80a1e685afe788
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached beers_utils-0.2.0-py3-none-any.whl
  Using cached camparee-0.4.2-py3-none-any.whl
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torch-2.5.1-cp311-cp311-ma

We then run the L1EM script as we made it with our changes on both practical and algorithmic ways.

In [ ]:
%%bash
rm -rf ./output/L1EM
# pushd ./L1EM/
# bash -e ./L1EM/run_L1EM_3.sh ./data/experiment/SRR3997504/SRR3997504_sorted.bam ./L1EM ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa
# popd
# pushd ./L1EM
# bash -e run_L1EM_3.sh /home/bio/git/l1emPipeline/output/beers2ashes/results/merged.sorted.bam /home/bio/git/l1emPipeline/L1EM /home/bio/git/l1emPipeline/data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ./output/L1EM
# popd
bash -e ./L1EM/run_L1EM_3.sh ./output/beers2ashes/results/merged.sorted.bam ./L1EM ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ./output/L1EM em
# python ./L1EM/caclmetrics.py -ic ./output/run_1/CAMPAREE/data/sample1/transcript_quantifications.txt -il ./L1EM/full_counts.txt -o ./output/metrics.txt -otemp ./output/merged.txt

STEP 1: realign


[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 0 reads
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa aln -k 3 -n 3 -t 12 -i 20 /home/bio/git/l1emPipeline/data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa unaligned.fq1
[main] Real time: 2.244 sec; CPU: 1.997 sec
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa aln -k 3 -n 3 -t 12 -i 20 /home/bio/git/l1emPipeline/data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa unaligned.fq2
[main] Real time: 2.511 sec; CPU: 1.998 sec
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa sampe /home/bio/git/l1emPipeline/data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa 1.sai 2.sai unaligned.fq1 unaligned.fq2
[main] Real time: 0.005 sec; CPU: 0.002 sec


STEP 2: extract


[bam_sort_core] merging from 0 files and 12 in-memory blocks...
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 89774 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 0 reads


STEP 3: candidate alignments


[bwa_aln_core] calculate SA coordinate... 24.38 sec
[bwa_aln_core] write to the disk... 0.02 sec
[bwa_aln_core] 3741 sequences have been processed.
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa aln -t 12 -N -n 3 -k 3 -i 20 -R 10000000 /home/bio/git/l1emPipeline/L1EM/annotation/L1EM.400.ens.noY.fa L1.fq1.aa
[main] Real time: 6.650 sec; CPU: 24.593 sec
[bwa_aln_core] calculate SA coordinate... 23.19 sec
[bwa_aln_core] write to the disk... 0.02 sec
[bwa_aln_core] 3741 sequences have been processed.
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa aln -t 12 -N -n 3 -k 3 -i 20 -R 10000000 /home/bio/git/l1emPipeline/L1EM/annotation/L1EM.400.ens.noY.fa L1.fq2.aa
[main] Real time: 5.813 sec; CPU: 23.404 sec
[bwa_aln_core] calculate SA coordinate... 24.87 sec
[bwa_aln_core] write to the disk... 0.03 sec
[bwa_aln_core] 3741 sequences have been processed.
[main] Version: 0.7.19-r1273
[main] CMD: /opt/anaconda3/envs/L1EM/bin/bwa aln -t 12 -N -n 

We finally calculate some metrics from the results of our custom L1EM script.

In [ ]:
%%bash
mkdir -p ./output/metricsAshes
rm -rf ./output/metricsAshes/*
python ./src/calculateMetrics.py -ic ./output/run_1/CAMPAREE/data/sample1/transcript_quantifications.txt -il ./output/L1EM/full_counts.txt -o ./output/metricsAshes/metrics.txt -otemp ./output/metricsAshes/merged.txt -method EM

/home/bio/git/l1emPipeline/./src/calculateMetrics.py:53: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['family.category.locus.strand'] = df['family.category.locus.strand'].str.replace(".", "_")
/home/bio/git/l1emPipeline/./src/calculateMetrics.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['family.category.locus.strand'] = df['family.category.locus.strand'].str.replace("+", "_")
/home/bio/git/l1emPipeline/./src/calculateMetrics.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['#transcript_id'] = df['#t

                                  locus  true_tpm         tpm
10399  L1PA2_0_chr5_163684073_163684126       NaN  117.194125
10400     L1HS_0_chr1_30567813_30568750       NaN   10.327296
10401     L1HS_0_chr1_41207751_41208441       NaN   15.305655
10402     L1HS_0_chr1_41854523_41855316       NaN   11.978680
10403     L1HS_0_chr1_41989723_41990366       NaN   11.043149
...                                 ...       ...         ...
10698  L1PA2_0_chrX_131503357_131505329       NaN    3.503166
10699  L1PA2_0_chrX_143003983_143004250       NaN    0.538070
10700  L1PA2_0_chrX_150131407_150133104       NaN    0.863739
10701   L1HS_0_chrX_152349286_152350533       NaN    5.511096
10702  L1PA3_0_chrX_152386378_152387309       NaN    7.050222

[304 rows x 3 columns]


We then calculate the L1EM results for the experiment SRR3997504 and its metrics.

In [ ]:
%%bash
rm -rf ./output/L1EMSRR
mkdir -p ./output/L1EMSRR
bash -e ./L1EM/run_L1EM_3.sh ./data/experiment/SRR3997504_sorted.bam ./L1EM ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ./output/L1EMSRR vb
mkdir -p ./output/metricsSRR
rm -rf ./output/metricsSRR/*
# python ./src/calculateMetrics.py -ic ./output/run_1/CAMPAREE/data/sample1/transcript_quantifications.txt -il ./output/L1EM/full_counts.txt -o ./output/metricsSRR/metrics.txt -otemp ./output/metricsSRR/merged.txt -method VB